In [ ]:
# Install PySpark via pip
!pip install pyspark

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.master("local[*]").appName("Colab Spark App").getOrCreate()

# Print the Spark version to confirm
print("Spark version:", spark.version)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
spark.sql("SELECT * FROM home_sales LIMIT 5").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_4br = spark.sql("""
  select year(date) AS Year, Round(AVG(price), 2) as Avg_Price
  from home_sales
  where bedrooms = 4
  Group By year(date)
  Order By Year
""")

avg_4br.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_3br = spark.sql("""
  select date_built AS Year, Round(AVG(price), 2) as Avg_Price
  from home_sales
  where bedrooms = 3 and bathrooms = 3
  Group By date_built
  Order By Year
""")

avg_3br.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_3br_2flr = spark.sql("""
  select date_built AS Year, Round(AVG(price), 2) as Avg_Price
  from home_sales
  where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
  Group By date_built
  Order By Year
""")

avg_3br_2flr.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_view_price = spark.sql("""
  select view as view, Round(AVG(price), 2) as Avg_Price
  from home_sales
  Group By view
  Having AVG(price) >= 350000
  Order By view DESC
""")

avg_view_price.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.3317687511444092 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_view_price = spark.sql("""
  select view as view, Round(AVG(price), 2) as Avg_Price
  from home_sales
  Group By view
  Having AVG(price) >= 350000
  Order By view DESC
""")

avg_view_price.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.6209254264831543 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")

In [20]:
# 11. Read the formatted parquet data.
df_date_built=spark.read.parquet('date_built_partitioned')

In [21]:
# 12. Create a temporary table for the parquet data.
df_date_built.createOrReplaceTempView('date_built_partition')

In [22]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_view_price = spark.sql("""
  select view as view, Round(AVG(price), 2) as Avg_Price
  from date_built_partition
  Group By view
  Having AVG(price) >= 350000
  Order By view DESC
""")

avg_view_price.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.1926867961883545 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [24]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False